# Parte 1: Aspectos Básicos de DataFrames en SparkSQL

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oramosul/abd-files/blob/main/spark/2-spark-sql/1-Aspectos-basicos-DataFrames.ipynb)

Documentación oficial de SparkSQL:
* https://spark.apache.org/docs/latest/sql-ref.html
* https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html
* https://spark.apache.org/docs/latest/api/python/index.html

In [ ]:
# Instalación de PySpark (si se está usando Google Colab, por ejemplo)
!pip install -q pyspark

**Creación de una sesión de Spark SQL**

Las aplicaciones de PySpark inician con una sesión de Spark llamada `SparkSession`, que es el punto de entrada a PySpark. Si Spark se corre en un shell (ejecutando: `pyspark`), no es necesario crear esta sesión de Spark dado que el shell crea la variable `spark` automáticamente. Si se corre un programa con `spark-submit` o si se tiene algún otro entorno (como Google Colab) sí es necesario iniciar la variable como se indica a continuación.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Si se desea especificar el nombre de la aplicación, se puede realizar añadiendo `appName`. Por ejemplo, si se desea el nombre `Aplicacion1`, se utiliza: `spark = SparkSession.builder.appName('Aplicacion1').getOrCreate()`

In [ ]:
type(spark)

## 1.&nbsp;Creación de un Data Frame

Un DataFrame de PySpark se puede crear usando `spark.createDataFrame`, que utiliza un argumento llamado `schema` para especificar el esquema del DataFrame. Si se omite el esquema, PySpark lo infiere a partir de una muestra de los datos utilizados.

Se puede crear DataFrames de varias maneras, como se muestra a continuación.

### 1.1. Creación por filas

In [3]:
from pyspark.sql import Row
from datetime import datetime, date

Creación usando `Row`, que especifica el esquema en cada fila. Notar que cada fila se trata como si fuera una tupla con nombre.

In [4]:
df = spark.createDataFrame([
    Row(ID=1, Valor=2., Nombre='Carlos', Fecha=date(2000, 1, 1), Hora=datetime(2000, 1, 1, 12, 0)),
    Row(ID=2, Valor=3., Nombre='María',  Fecha=date(2000, 2, 1), Hora=datetime(2000, 1, 2, 12, 0)),
    Row(ID=4, Valor=5., Nombre='Pedro',  Fecha=date(2000, 3, 1), Hora=datetime(2000, 1, 3, 12, 0))
])

# Mostrar el DataFrame
df.show()

+---+-----+------+----------+-------------------+
| ID|Valor|Nombre|     Fecha|               Hora|
+---+-----+------+----------+-------------------+
|  1|  2.0|Carlos|2000-01-01|2000-01-01 12:00:00|
|  2|  3.0| María|2000-02-01|2000-01-02 12:00:00|
|  4|  5.0| Pedro|2000-03-01|2000-01-03 12:00:00|
+---+-----+------+----------+-------------------+



In [5]:
# Mostrar el esquema del DataFrame
df.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Valor: double (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Hora: timestamp (nullable = true)



Creación usando tuplas y especificando el esquema de manera explícita (con el tipo de dato).

In [6]:
df = spark.createDataFrame([
    (1, 2., 'Carlos', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'María',  date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (4, 5., 'Pedro',  date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='ID long, Valor double, Nombre string, Fecha date, Hora timestamp')

df.show()

+---+-----+------+----------+-------------------+
| ID|Valor|Nombre|     Fecha|               Hora|
+---+-----+------+----------+-------------------+
|  1|  2.0|Carlos|2000-01-01|2000-01-01 12:00:00|
|  2|  3.0| María|2000-02-01|2000-01-02 12:00:00|
|  4|  5.0| Pedro|2000-03-01|2000-01-03 12:00:00|
+---+-----+------+----------+-------------------+



### 1.2. Creación usando un DataFrame de Pandas

Documentación del DataFrame de Pandas: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [7]:
import pandas as pd

In [8]:
# Creación del DataFrame de Pandas
pandas_df = pd.DataFrame({
    'ID': [1, 2, 3],
    'Valor': [2., 3., 4.],
    'Nombre': ['Carlos', 'María', 'Pedro'],
    'Fecha': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'Hora': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})

# Creación del DataFrame de Spark a partir del DataFrame de Pandas
df = spark.createDataFrame(pandas_df)

# Mostrar el DataFrame
df.show()

+---+-----+------+----------+-------------------+
| ID|Valor|Nombre|     Fecha|               Hora|
+---+-----+------+----------+-------------------+
|  1|  2.0|Carlos|2000-01-01|2000-01-01 12:00:00|
|  2|  3.0| María|2000-02-01|2000-01-02 12:00:00|
|  3|  4.0| Pedro|2000-03-01|2000-01-03 12:00:00|
+---+-----+------+----------+-------------------+



### 1.3. Creación a partir de un RDD

Si el RDD no tiene esquema, se le tiene que asignar al momento de crear el DataFrame usando `schema`, donde se incluye el nombre del campo y el tipo de dato (string, int, long, double, date, timestamp)

In [9]:
sc = spark.sparkContext

# Creación de un RDD
rdd = sc.parallelize([("Juan", 20), ("María", 25), ("Victoria", 22)])

# Creación de DataFrame a partir del RDD
df = spark.createDataFrame(rdd,
                           schema='nombre string,edad int')
df.show()

+--------+----+
|  nombre|edad|
+--------+----+
|    Juan|  20|
|   María|  25|
|Victoria|  22|
+--------+----+



Se le puede asignar esquema al RDD definiendo cada elemento como una fila `Row`, que es similar a una tupla con nombre.

In [10]:
from pyspark.sql import Row

rdd = sc.parallelize([Row(nombre="Juan", edad=20),
                      Row(nombre="Kiara", edad=19),
                      Row(nombre="Keiko", edad=80)])

df = spark.createDataFrame(rdd)
df.show()

+------+----+
|nombre|edad|
+------+----+
|  Juan|  20|
| Kiara|  19|
| Keiko|  80|
+------+----+



## 2.&nbsp;Carga de Datos a un DataFrame

Para más información sobre los formatos mencionados aquí, u otros formatos, consultar estas referencias:
* https://spark.apache.org/docs/latest/sql-data-sources.html
* https://github.com/apache/spark/blob/master/examples/src/main/python/sql/datasource.py

Por facilidad se cargará archivos del repositorio usando `wget`. Estos archivos (o carpetas) también pueden ser subidos manualmente a Google Colab. En caso de estar trabajando en un clúster con HDFS, estos archivos deberían estar en el clúster.

In [18]:
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/personas.csv
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/recursos.zip
!unzip -q recursos.zip
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/texto-personas.txt
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/personas.json
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/personas.parquet

### 2.1. Lectura de CSV

Más información: https://spark.apache.org/docs/latest/sql-data-sources-csv.html

Lectura directa sin especificar la estructura. Por defecto asume que el separador es una coma `,`. En este ejemplo el delimitador es `;`, así que al leer no se separa las columnas sino se lee como una sola columna (con nombre `_c0` por defecto)

In [12]:
df = spark.read.csv("personas.csv")
df.show()

+--------------------+
|                 _c0|
+--------------------+
| nombre;edad;trabajo|
|Jorge;30;Desarrol...|
|  María;23;Psicóloga|
|    Pedro;25;Abogado|
|Jennifer;21;Admin...|
|Mariana;23;Influe...|
|  Adalí;24;Estilista|
+--------------------+



Lectura especificando un delimitador. En este ejemplo, el delimitador es `;`. Los nombres de las columnas se asignan por defecto como `_c0`, `_c1`, `_c2`.

In [13]:
df = spark.read.option("delimiter", ";").csv("personas.csv")
df.show()

+--------+----+--------------+
|     _c0| _c1|           _c2|
+--------+----+--------------+
|  nombre|edad|       trabajo|
|   Jorge|  30| Desarrollador|
|   María|  23|     Psicóloga|
|   Pedro|  25|       Abogado|
|Jennifer|  21|Administradora|
| Mariana|  23|    Influencer|
|   Adalí|  24|     Estilista|
+--------+----+--------------+



Lectura especificando un delimitador y un encabezado (primera fila del CSV).

In [14]:
df = spark.read.options(delimiter=";", header=True).csv("personas.csv")

# También se podría especificar cada opción por separado:
# df = spark.read.option("delimiter", ";").option("header", True).csv("personas.csv")

df.show()

+--------+----+--------------+
|  nombre|edad|       trabajo|
+--------+----+--------------+
|   Jorge|  30| Desarrollador|
|   María|  23|     Psicóloga|
|   Pedro|  25|       Abogado|
|Jennifer|  21|Administradora|
| Mariana|  23|    Influencer|
|   Adalí|  24|     Estilista|
+--------+----+--------------+



In [15]:
# Otra alternativa de lectura es la siguiente:
df = spark.read.load("personas.csv",
                     format="csv", sep=";",
                     inferSchema="true", header="true")

df.show()

+--------+----+--------------+
|  nombre|edad|       trabajo|
+--------+----+--------------+
|   Jorge|  30| Desarrollador|
|   María|  23|     Psicóloga|
|   Pedro|  25|       Abogado|
|Jennifer|  21|Administradora|
| Mariana|  23|    Influencer|
|   Adalí|  24|     Estilista|
+--------+----+--------------+



Lectura de varios archivos CSV en una carpeta. Se debe asegurar de que solo existan archivos CSV en la carpeta y que el formato sea similar, para que puedan ser leidos y cargados adecuadamente.

In [19]:
# Nombre de la carpeta donde se encuentran varios CSV con similar estructura
carpetaDatos = "recursos"

# Lectura de varios csv en un solo DataFrame especificando la carpeta
df = spark.read.option("delimiter", ";").option("header", True).csv("recursos")
df.show()

+-------+----+--------------+
| nombre|edad|       trabajo|
+-------+----+--------------+
| Carlos|  24|     Ingeniero|
|Mariana|  22|     Contadora|
| Sharon|  28|Administradora|
|  Jorge|  30| Desarrollador|
|  María|  32|     Ingeniero|
|  Pedro|  25|       Abogado|
|  Mario|  25|       Abogado|
| Salomé|  22|        Modelo|
| Fershi|  19|    Influencer|
+-------+----+--------------+



### 2.2. Lectura de TXT

Más información: https://spark.apache.org/docs/latest/sql-data-sources-text.html

In [21]:
df = spark.read.text("texto-personas.txt")
df.show()

+-------------+
|        value|
+-------------+
|   Maikol, 29|
|   Andrés, 30|
|Justencio, 19|
+-------------+



Se puede indicar el separador de línea (fila) usando la opción `lineSep`. Por defecto se asume `\n`, `\r\n` o `\r`. Para el siguiente ejemplo se
forzará el uso de la `,` como si fuera un separador de línea.

In [22]:
df = spark.read.text("texto-personas.txt", lineSep=",")
df.show()

+--------------+
|         value|
+--------------+
|        Maikol|
|    29\nAndrés|
| 30\nJustencio|
|            19|
+--------------+



### 2.3. Lectura de JSON

Más información: https://spark.apache.org/docs/latest/sql-data-sources-json.html

Notar que el archivo JSON no es un típico JSON, sino que cada línea debe contener un objeto JSON válido, separado y auto contenido. A este formato se le suele llamar "JSON Lines text format" o "newline-delimited JSON" (https://jsonlines.org/).

In [23]:
df = spark.read.json("personas.json")

df.show()

+----+--------+------+
|edad|estatura|nombre|
+----+--------+------+
|NULL|    NULL| Pedro|
|  30|    NULL| María|
|  25|    NULL|Carlos|
|NULL|     162|Teresa|
+----+--------+------+



In [24]:
# Esquema inferido
df.printSchema()

root
 |-- edad: long (nullable = true)
 |-- estatura: long (nullable = true)
 |-- nombre: string (nullable = true)



De manera alternativa, se puede crear un DataFrame utilizando un RDD que contiene objetos JSON como cadenas de caracteres.

In [25]:
datos = ['{"nombre":"Juan","direccion":{"ciudad":"Lima","distrito":"Miraflores"}}',
         '{"nombre":"Camila","direccion":{"ciudad":"Arequipa","distrito":"Cercado"}}']

sc = spark.sparkContext
rdd = sc.parallelize(datos)
rdd.collect()

['{"nombre":"Juan","direccion":{"ciudad":"Lima","distrito":"Miraflores"}}',
 '{"nombre":"Camila","direccion":{"ciudad":"Arequipa","distrito":"Cercado"}}']

In [26]:
df = spark.read.json(rdd)
df.show()

+-------------------+------+
|          direccion|nombre|
+-------------------+------+
| {Lima, Miraflores}|  Juan|
|{Arequipa, Cercado}|Camila|
+-------------------+------+



### 2.4. Lectura de Parquet

Más información: https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

Parquet es un formato columnar soportado por muchos sistemas de procesamiento y análisis de datos para Big Data. Spark SQL al leer archivos Parquet preserva el esquema de los datos originales.

In [27]:
df = spark.read.load("personas.parquet")

df.show()

+------+----+-------------+
|nombre|edad|      trabajo|
+------+----+-------------+
| Jorge|  30|Desarrollador|
| María|  32|    Ingeniero|
| Pedro|  25|      Abogado|
+------+----+-------------+



## 3.&nbsp;Visualización de Datos

In [28]:
# Lectura de un dataframe de un conjunto de CSVs
df = spark.read.load("recursos", format="csv", sep=";",
                     inferSchema="true", header="true")

Si se desea visualizar el contenido del dataframe escribiendo únicamente el nombre del DataFrame (en un cuaderno de Jupyter) se puede activar la opción `spark.sql.repl.eagerEval.enabled`. Para todo lo demás, no es necesario activar esta opción.

In [29]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

**Visualización del DataFrame**

In [30]:
# Si se activa la opción anterior, se puede usar el nombre del dataframe solito
df

nombre,edad,trabajo
Carlos,24,Ingeniero
Mariana,22,Contadora
Sharon,28,Administradora
Jorge,30,Desarrollador
María,32,Ingeniero
Pedro,25,Abogado
Mario,25,Abogado
Salomé,22,Modelo
Fershi,19,Influencer


Visualización de una determinada cantidad de filas usando `show(N)`, donde `N` es el número de filas. Si no se especifica, por defecto muestra las 20 primeras filas.

In [31]:
df.show(5)

+-------+----+--------------+
| nombre|edad|       trabajo|
+-------+----+--------------+
| Carlos|  24|     Ingeniero|
|Mariana|  22|     Contadora|
| Sharon|  28|Administradora|
|  Jorge|  30| Desarrollador|
|  María|  32|     Ingeniero|
+-------+----+--------------+
only showing top 5 rows



Las filas pueden ser mostradas de manera vertical con la opción `vertical`. Esta forma de visualizar suele ser útil cuando las filas son muy largas para ser mostradas horizontalmente.

In [32]:
df.show(2, vertical=True)

-RECORD 0------------
 nombre  | Carlos    
 edad    | 24        
 trabajo | Ingeniero 
-RECORD 1------------
 nombre  | Mariana   
 edad    | 22        
 trabajo | Contadora 
only showing top 2 rows



**Esquema del DataFrame**

Se puede visualizar el nombre de las columnas y la estructura de la tabla usando `columns` y `printSchema`.

In [33]:
# Nombres de columnas
df.columns

['nombre', 'edad', 'trabajo']

In [34]:
# Mostrar esquema
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- trabajo: string (nullable = true)



**Resumen de los datos**

In [35]:
df.describe().show()

+-------+------+------------------+-------+
|summary|nombre|              edad|trabajo|
+-------+------+------------------+-------+
|  count|     9|                 9|      9|
|   mean|  NULL| 25.22222222222222|   NULL|
| stddev|  NULL|4.1466184348749096|   NULL|
|    min|Carlos|                19|Abogado|
|    max|Sharon|                32| Modelo|
+-------+------+------------------+-------+



## 4.&nbsp;Selección y acceso a los datos

Los DataFrames se evalúan de manera ociosa ("lazy") y cuando se selecciona una columna no se realiza el cálculo sino que solo se retorna una instancia de tipo `Column`.


In [36]:
df.nombre

Column<'nombre'>

### 4.1. Selección de columnas

Para el acceso a una columna o varias columnas se utiliza `DataFrame.select()`.

In [37]:
df.select('edad').show(3)

+----+
|edad|
+----+
|  24|
|  22|
|  28|
+----+
only showing top 3 rows



In [38]:
df.select(['nombre', 'edad']).show(3)

+-------+----+
| nombre|edad|
+-------+----+
| Carlos|  24|
|Mariana|  22|
| Sharon|  28|
+-------+----+
only showing top 3 rows



In [39]:
df.select("nombre", "trabajo").show(3)

+-------+--------------+
| nombre|       trabajo|
+-------+--------------+
| Carlos|     Ingeniero|
|Mariana|     Contadora|
| Sharon|Administradora|
+-------+--------------+
only showing top 3 rows



### 4.2. Recolección de datos

`DataFrame.collect()` recolecta los datos distribuidos y los envía al driver como datos locales en Python. Esto puede generar errores de falta de memoria cuando el dataset es muy grande para caber en el driver, ya que se recolecta todos los datos de los ejecutores hacia el driver.

In [40]:
df.collect()

[Row(nombre='Carlos', edad=24, trabajo='Ingeniero'),
 Row(nombre='Mariana', edad=22, trabajo='Contadora'),
 Row(nombre='Sharon', edad=28, trabajo='Administradora'),
 Row(nombre='Jorge', edad=30, trabajo='Desarrollador'),
 Row(nombre='María', edad=32, trabajo='Ingeniero'),
 Row(nombre='Pedro', edad=25, trabajo='Abogado'),
 Row(nombre='Mario', edad=25, trabajo='Abogado'),
 Row(nombre='Salomé', edad=22, trabajo='Modelo'),
 Row(nombre='Fershi', edad=19, trabajo='Influencer')]

Para evitar la excepción de falta de memoria, se puede usar `DataFrame.take(N)`, `DataFrame.head(N)` o `DataFrame.tail(N)`, donde `N` es el número de filas.

In [41]:
df.take(3)

[Row(nombre='Carlos', edad=24, trabajo='Ingeniero'),
 Row(nombre='Mariana', edad=22, trabajo='Contadora'),
 Row(nombre='Sharon', edad=28, trabajo='Administradora')]

In [42]:
df.head(3)

[Row(nombre='Carlos', edad=24, trabajo='Ingeniero'),
 Row(nombre='Mariana', edad=22, trabajo='Contadora'),
 Row(nombre='Sharon', edad=28, trabajo='Administradora')]

In [43]:
df.tail(3)

[Row(nombre='Mario', edad=25, trabajo='Abogado'),
 Row(nombre='Salomé', edad=22, trabajo='Modelo'),
 Row(nombre='Fershi', edad=19, trabajo='Influencer')]

La extracción de datos para procesarlos "manualmente" se puede realizar extrayendo las filas `Row` a través de índices.

In [44]:
# Extraccion de filas
df.head(2)[1]

Row(nombre='Mariana', edad=22, trabajo='Contadora')

In [45]:
# Recuperación de valores
dato = df.head(2)[1]
dato.edad     # Equivalente a dato[0]

22

In [46]:
# Se puede convertir el dato en un diccionario
dato = df.head(2)[1].asDict()
print(dato)

{'nombre': 'Mariana', 'edad': 22, 'trabajo': 'Contadora'}


### 4.3. Creación de nuevas columnas

Para crear una nueva columna se utiliza `withColumn`. Notar que el DataFrame es inmutable: si se desea almacenar el resultado se debe crear un nuevo DataFrame.

In [47]:
# Crear una nueva columna de año de nacimiento (asumiendo que es el año 2023)
df2 = df.withColumn('año nacimiento', 2023-df['edad'])

df2.show(3)

+-------+----+--------------+--------------+
| nombre|edad|       trabajo|año nacimiento|
+-------+----+--------------+--------------+
| Carlos|  24|     Ingeniero|          1999|
|Mariana|  22|     Contadora|          2001|
| Sharon|  28|Administradora|          1995|
+-------+----+--------------+--------------+
only showing top 3 rows



In [48]:
from pyspark.sql.functions import upper

# Crear una nueva columna con la columna "nombre" en mayúsculas
df.withColumn('nombre_may', upper(df['nombre'])).show(3)

+-------+----+--------------+----------+
| nombre|edad|       trabajo|nombre_may|
+-------+----+--------------+----------+
| Carlos|  24|     Ingeniero|    CARLOS|
|Mariana|  22|     Contadora|   MARIANA|
| Sharon|  28|Administradora|    SHARON|
+-------+----+--------------+----------+
only showing top 3 rows



Se puede cambiar el nombre de una columna utilizando `withColumnRenamed`. Notar que para almacenar el nombre cambiado se debe crear un nuevo DataFrame.

In [49]:
# Cambiar nombre a una columna
df2 = df.withColumnRenamed('edad', 'años')

df2.show(3)

+-------+----+--------------+
| nombre|años|       trabajo|
+-------+----+--------------+
| Carlos|  24|     Ingeniero|
|Mariana|  22|     Contadora|
| Sharon|  28|Administradora|
+-------+----+--------------+
only showing top 3 rows



## 5.&nbsp;Modificación del esquema

Si Spark no infiere adecuadamente los datos, o si se desea modificar el tipo de dato de las columnas, se puede crear un nuevo esquema especificando los tipos de datos.

Se puede ver los tipos de datos existentes en: https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [50]:
from pyspark.sql.types import StructField, StructType

# Importar los tipos de datos que se va a utilizar
from pyspark.sql.types import StringType, FloatType

En este ejemplo se intenta modificar lo siguiente:
* `edad` con tipo float
* cambiar el nombre del tercer campo de "trabajo" a "ocupación"

In [51]:
# Creación del nuevo esquema
campos = [StructField('nombre', StringType(), True),
          StructField('edad', FloatType(), True),
          StructField('ocupación', StringType(), True),
         ]
esquema2 = StructType(fields = campos)

In [52]:
# Lectura de los datos asignando el nuevo esquema
df2 = spark.read.load("recursos", format="csv", sep=";",
                     schema=esquema2, header="true")

# Mostrar el nuevo esquema
df2.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- ocupación: string (nullable = true)



In [53]:
df2.show(3)

+-------+----+--------------+
| nombre|edad|     ocupación|
+-------+----+--------------+
| Carlos|24.0|     Ingeniero|
|Mariana|22.0|     Contadora|
| Sharon|28.0|Administradora|
+-------+----+--------------+
only showing top 3 rows



## 6.&nbsp;Almacenamiento del DataFrame

### 6.1. Conversión a DataFrame de Pandas

Un DataFrame de Spark puede convertirse a un DataFrame de Pandas, para poder utilizar el API de pandas (y eventualmente poder hacer figuras y demás). Notar que `toPandas` recolecta todos los datos al driver, de tal modo que puede fácilmente causar errores de tamaño de memoria en el driver cuando se tiene muchos datos.

In [54]:
df_pandas = df.toPandas()

In [55]:
df_pandas.head(3)

,nombre,edad,trabajo
0,Carlos,24,Ingeniero
1,Mariana,22,Contadora
2,Sharon,28,Administradora


### 6.2. Almacenamiento en disco

Al almacenar como CSV se puede especificar las mismas opciones que para lectura (por ejemplo, `header`). Se asigna el nombre de una carpeta, la cual contendrá un archivo vacío llamado `_SUCCESS`, que indica el éxito de la grabación, así como múltiples archivos csv.

In [56]:
df.write.options(header=True).csv("salida_csv")

La salida también puede ser parquet. Igualmente, se indica el nombre de la carpeta de salida.

In [57]:
df.write.parquet("salida_parquet")

Otro formato de salida es ORC.

In [58]:
df.write.orc("salida_orc")